In [2]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [3]:
# INPUT 
data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Qty 4 Months/Input.xlsx",sheet_name="Input").rename(columns={"FOB Price (USD/Pcs)":"Unit Price"})
# Exchange Rate
data_exchange_rate=pd.read_excel("../VACP/Python NCC Min/Vietcombank Exchange Currency.xlsx",usecols="B,E").rename(columns={"Giá bán VND":"Tỷ giá VND"})

# # Tinh Unit Price VND
data_input=data_input.merge(data_exchange_rate,on="Mã nguyên tệ")
Exchange_Cost=0.05

data_input["Unit Price VND"] = np.round((1+Exchange_Cost)*data_input["Unit Price"]*data_input["Tỷ giá VND"],0)
data_input=data_input.drop(columns=["No.","Tỷ giá VND"])
data_input.head()


,Brand,Order Conf. Date,Calculated Date,Order Ref.,PO Number,ID,Substitution,Description,Order Qty,Unit Price,Mã nguyên tệ,Total Order Amount,Agt 22 Ready Stock,Số lượng cần nhập,Số lượng chênh lệch,Note,Unit Price VND
0,Denso,2022-01-01,2022-08-22,NaN,NaN,065000-2940ND AC,NaN,Motor Assy,20,31.85,USD,637.00,0.0,20.0,0.0,Order,803623.0
1,Denso,2022-01-01,2022-08-22,NaN,NaN,168000-1070ND AC,NaN,Motor Fan,20,18.35,USD,367.00,0.0,20.0,0.0,Order,462998.0
2,Denso,2022-01-01,2022-08-22,NaN,NaN,291000-9011 ND,NaN,Fuel Pump,15,24.99,USD,374.85,0.0,20.0,-5.0,Order,630535.0
3,Denso,2022-01-01,2022-08-22,NaN,NaN,291000-9011 ND,NaN,Fuel Pump,20,24.99,USD,499.80,0.0,20.0,0.0,Order,630535.0
4,Toyota,2022-04-26,2022-08-22,NaN,NaN,13011-0L030,NaN,"RING SET, PISTON",4,45.83,USD,183.32,0.0,4.0,0.0,Order,1156360.0


In [4]:
# NHAP - XUAT - TON  * GIA BAN * DANH MUC HANG HOA

data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" }).dropna(subset=['ID']).reset_index(drop=True)


data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6).rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                }).dropna(subset=['ID']).reset_index(drop=True)
                                
data_giaban=pd.read_excel("../VACP/Python NCC Min\Báo cáo giá bán, tồn kho.xls",usecols="C,F",skipfooter=1,header=4).rename(columns={"Mã số":"ID"}).dropna(
                subset=['ID']).reset_index(drop=True)

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M").rename(columns={
                                "Unnamed: 0":"ID Nhóm hàng",
                                "Unnamed: 2":"ID",
                                "Unnamed: 3":"Tên vật tư",
                                "Unnamed: 12":"Hãng"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)



C:\Users\TAM\AppData\Local\Temp\ipykernel_7712\1085106072.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={


In [5]:
# Tạo ID Date Ngày bán
data_banhang["Năm"]=data_banhang["Ngày bán"].dt.year
data_banhang["Tháng"]=data_banhang["Ngày bán"].dt.month
data_banhang["ID Date"]=data_banhang["Ngày bán"].dt.year.astype(str)+"-"+data_banhang["Ngày bán"].dt.month.astype(str)


pivot_banhang_thang=data_banhang.groupby(['ID','Năm','Tháng'],as_index=False)["Tổng bán"].sum()

#Tính trung bình Tổng bán tháng
pivot_banhang_tb=pivot_banhang_thang.groupby(["ID"],as_index=False).agg(
    {
        'Tổng bán':'sum',
        'Tháng':'count'
    }
)
pivot_banhang_tb["Tổng bán trung bình tháng"]=pivot_banhang_tb["Tổng bán"]/pivot_banhang_tb["Tháng"]
pivot_banhang_tb=pivot_banhang_tb.sort_values("Tổng bán trung bình tháng",ascending=False).reset_index(drop=True)
pivot_banhang_tb.head()


,ID,Tổng bán,Tháng,Tổng bán trung bình tháng
0,90919-T1002,25505,10,2550.5
1,90919-T1004,22201,10,2220.1
2,48520-8Z125,5887,10,588.7
3,48510-8Z265,5759,10,575.9
4,JK272000-6900,5678,10,567.8


In [6]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =np.round(
                                         (1+TransCost)*np.where(data_nhapxuatton["Giá trị xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Giá trị tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            ))
                                    ,0)

In [7]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
 

In [8]:
# Gộp dữ liệu
data_nhap_ban=data_nhapxuatton.merge(
    data_danhmuchanghoa[["ID","Hãng"]],on="ID").merge(
        data_banhang_latest[["ID","Ngày bán","Giá bán giảm"]],on='ID',how='left').merge(
            data_giaban,on="ID").merge(
                pivot_banhang_tb[["ID","Tổng bán","Tổng bán trung bình tháng"]],on="ID",how='left'
            )

# TÍnh số tháng tồn trong kho


data_nhap_ban["Số tháng tồn trong kho"]=np.round(
                                        (363/30)/data_nhap_ban["Giá vốn"]/(np.where(
                                        # Nếu Tồn dương và đầu kỳ dương thì lấy Giá vốn / (TB Giá trị đầu kỳ + nhập + Cuối kỳ)
                                                        (data_nhap_ban["Giá trị đầu kỳ"]>0) & (data_nhap_ban["Giá trị tồn"]>0),
                                                                np.mean([data_nhap_ban["Tổng đầu kỳ"],data_nhap_ban["Tổng tồn"]])/
                                                                    np.mean([data_nhap_ban["Giá trị đầu kỳ"],data_nhap_ban["Giá trị tồn"]]),-999)),1)
                                
data_report=data_input.merge(data_nhap_ban,on="ID",how="left")
# data_report=data_input.merge(data_nhap_ban[["ID","Giá vốn","Giá bán","Giá bán giảm","Tổng đầu kỳ","Tổng nhập","Tổng xuất","Tổng tồn","Số tháng tồn trong kho"]], on="ID", how="left")



In [9]:
data_report_show=data_report[["ID",
    "Order Conf. Date",
    "Calculated Date",
    "Unit Price VND",
    "Giá vốn",
    "Giá bán giảm", 
    "Order Qty",
    "Số lượng cần nhập", 
    "Agt 22 Ready Stock",
    "Tổng đầu kỳ",
    "Tổng nhập",
    "Tổng xuất",
    "Tổng tồn",
    "Giá trị đầu kỳ", 
    "Giá trị nhập",
    "Giá trị xuất", 
    "Giá trị tồn",
    "Tổng bán",
    "Tổng bán trung bình tháng",
    "Số tháng tồn trong kho"]].sort_values(by=["Số tháng tồn trong kho","Tổng tồn","ID"],ascending=False).reset_index(drop=True)

In [10]:
data_report_show

,ID,Order Conf. Date,Calculated Date,Unit Price VND,Giá vốn,Giá bán giảm,Order Qty,Số lượng cần nhập,Agt 22 Ready Stock,Tổng đầu kỳ,Tổng nhập,Tổng xuất,Tổng tồn,Giá trị đầu kỳ,Giá trị nhập,Giá trị xuất,Giá trị tồn,Tổng bán,Tổng bán trung bình tháng,Số tháng tồn trong kho
0,90919-T1004,2022-06-07,2022-08-22,20185.0,20702.0,25000.0,10000,10000.0,560.0,7588.0,34200.0,39211.0,2577.0,154632887.0,6.695384e+08,7.730861e+08,51085208.0,22201.0,2220.100000,187.3
1,90385-T0004,2022-06-07,2022-08-22,16400.0,21440.0,40000.0,250,NaN,250.0,117.0,450.0,276.0,291.0,2185792.0,8.166394e+06,5.635731e+06,4716455.0,256.0,25.600000,180.8
2,90080-91232,2022-06-07,2022-08-22,42137.0,34612.0,100000.0,20,20.0,0.0,5.0,28.0,24.0,9.0,207975.0,8.593720e+05,7.911230e+05,276224.0,19.0,2.375000,112.0
3,08889-80280,2022-06-07,2022-08-22,147352.0,151673.0,200000.0,1500,10000.0,1200.0,718.0,10084.0,7182.0,3620.0,103624843.0,1.457486e+09,1.037442e+09,523668954.0,4909.0,545.444444,25.6
4,JK272000-3380,2022-02-17,2022-08-22,180153.0,174329.0,240000.0,500,NaN,200.0,158.0,199.0,353.0,4.0,25436277.0,3.385199e+07,5.860782e+07,680442.0,353.0,117.666667,22.2
5,11115-0Y030,NaT,2022-08-22,254838.0,256851.0,400000.0,10,NaN,NaN,15.0,26.0,38.0,3.0,3669476.0,6.358873e+06,9.295577e+06,732772.0,23.0,2.555556,15.1
6,90916-T2039,2022-06-07,2022-08-22,230868.0,274594.0,430000.0,500,1000.0,162.0,633.0,803.0,1321.0,115.0,181991846.0,1.943710e+08,3.454658e+08,30897011.0,923.0,92.300000,14.1
7,48609-0K020,2022-06-07,2022-08-22,287639.0,304531.0,400000.0,20,20.0,0.0,32.0,42.0,65.0,9.0,9529704.0,1.195558e+07,1.885193e+07,2633352.0,45.0,5.625000,12.7
8,04465-YZZZ1,NaT,2022-08-22,336588.0,325461.0,450000.0,10,NaN,NaN,2.0,119.0,78.0,43.0,600292.0,3.690519e+07,2.417709e+07,13328388.0,18.0,2.571429,11.9
9,90916-T2024,2022-06-07,2022-08-22,261398.0,355480.0,450000.0,1000,1000.0,0.0,440.0,1115.0,1518.0,37.0,163244052.0,3.632047e+08,5.139222e+08,12526508.0,1517.0,151.700000,10.9


In [ ]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Python NCC Min/Analysing Qty 4 Months/Report.xlsx') as writer:
    data_report_show.to_excel(writer,sheet_name="Tổng hợp")